# **Preparación de los archivos**

In [1]:

!pip install trimesh plotly mapbox_earcut scipy pyvista

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')



ValueError: mount failed

In [ ]:
ruta_obj = "/content/drive/Shared drives/POLYTOPES/TRIANGULATIONS/truncated dodecahedron.obj"


In [ ]:
import os
import trimesh

# Verifica si existe
print("¿Archivo encontrado?", os.path.exists(ruta_obj))

# Cargar la malla
mesh = trimesh.load(ruta_obj)

# Ver resumen
print(mesh)


¿Archivo encontrado? True
<trimesh.Trimesh(vertices.shape=(60, 3), faces.shape=(116, 3))>


# **Vista general**

In [ ]:

import plotly.graph_objects as go

In [ ]:

def mostrar_malla(mesh, titulo="Triangulación"):
    v = mesh.vertices
    f = mesh.faces
    fig = go.Figure(data=[go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        opacity=0.5, color='lightblue'
    )])
    fig.update_layout(title=titulo, scene=dict(aspectmode='data'))
    fig.show()

# **Visualización coloreada**

In [ ]:

import pyvista as pv
import numpy as np
import plotly.graph_objects as go
import trimesh

In [ ]:

def visualizar_coloreado(mesh, titulo="Malla Triangulada"):
    v = mesh.vertices
    f = mesh.faces

    fig = go.Figure()

    # Dar un color distinto a cada triángulo
    colores = np.random.rand(len(f), 3)

    for i, tri in enumerate(f):
        tri_vertices = v[tri]
        fig.add_trace(go.Mesh3d(
            x=tri_vertices[:, 0],
            y=tri_vertices[:, 1],
            z=tri_vertices[:, 2],
            i=[0], j=[1], k=[2],
            color=f'rgb({colores[i][0]*255:.0f}, {colores[i][1]*255:.0f}, {colores[i][2]*255:.0f})',
            opacity=1.0,
            name=f"Face {i}"
        ))

    fig.update_layout(title=titulo, scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=30))
    fig.show()

In [ ]:
def visualizar_transparente_con_bordes(mesh, titulo="Interior visible"):
    import plotly.graph_objects as go

    v = mesh.vertices
    f = mesh.faces

    # Crear malla transparente
    fig = go.Figure(data=[go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        color='lightblue', opacity=0.3,
        name="Poliedro"
    )])

    # Añadir líneas de borde
    for face in f:
        puntos = v[face]
        # Conectar los 3 lados del triángulo
        edges = [(0,1), (1,2), (2,0)]
        for a,b in edges:
            fig.add_trace(go.Scatter3d(
                x=[puntos[a][0], puntos[b][0]],
                y=[puntos[a][1], puntos[b][1]],
                z=[puntos[a][2], puntos[b][2]],
                mode='lines',
                line=dict(color='black', width=2),
                showlegend=False
            ))

    fig.update_layout(title=titulo, scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=30))
    fig.show()


In [ ]:
def tetrahedralizar_y_extraer_superficie(mesh):
    import pyvista as pv
    import numpy as np
    from trimesh import Trimesh

    faces_pv = np.hstack([[3, *face] for face in mesh.faces])
    pv_mesh = pv.PolyData(mesh.vertices, faces_pv)

    # Relleno interno
    volumen = pv_mesh.delaunay_3d()

    # Extraer superficie
    superficie = volumen.extract_surface()
    v = np.array(superficie.points)
    f = np.array(superficie.faces).reshape((-1, 4))[:, 1:]

    return Trimesh(vertices=v, faces=f)


In [ ]:
Primera = visualizar_coloreado(mesh, "Original")
Primera

In [ ]:
visualizar_transparente_con_bordes(mesh, "Interior del Poliedro")

In [ ]:

# Convertir caras a formato PyVista
faces_pv = np.hstack([[3, *face] for face in mesh.faces])
pv_mesh = pv.PolyData(mesh.vertices, faces_pv)

# Crear volumen tetraedral
volumen = pv_mesh.delaunay_3d()

# Visualizar con superficie semitransparente
superficie = volumen.extract_surface()
v = np.array(superficie.points)
f = np.array(superficie.faces).reshape((-1, 4))[:, 1:]

mesh_tetra = trimesh.Trimesh(vertices=v, faces=f)
visualizar_transparente_con_bordes(mesh_tetra, "Tetrahedralización (volumen interno)")

# **Triangulación por subdivisión**

In [ ]:
def tetrahedralizar_y_extraer_superficie(mesh):
    import pyvista as pv
    import numpy as np
    from trimesh import Trimesh

    faces_pv = np.hstack([[3, *face] for face in mesh.faces])
    pv_mesh = pv.PolyData(mesh.vertices, faces_pv)

    # Relleno interno
    volumen = pv_mesh.delaunay_3d()

    # Extraer superficie
    superficie = volumen.extract_surface()
    v = np.array(superficie.points)
    f = np.array(superficie.faces).reshape((-1, 4))[:, 1:]

    return Trimesh(vertices=v, faces=f)


In [ ]:
import os
import trimesh

def exportar_triangulos(mesh, carpeta="triangulos_exportados"):
    os.makedirs(carpeta, exist_ok=True)
    for idx, face in enumerate(mesh.faces):
        vertices = mesh.vertices[face]
        pieza = trimesh.Trimesh(vertices=vertices, faces=[[0,1,2]])
        pieza.export(f"{carpeta}/triangulo_{idx:03}.stl")

In [ ]:
triang_1 = mesh.subdivide()
triang_2 = mesh.subdivide_to_size(max_edge=0.3)
triang_3 = mesh.convex_hull
triang_4 = tetrahedralizar_y_extraer_superficie(mesh)

In [ ]:
# Ver triangulación coloreada
visualizar_coloreado(triang_1, "Triangulación 1")

# Ver interior con transparencia
visualizar_transparente_con_bordes(triang_1, "Interior 1")

# Tetrahedralizar y visualizar
malla_tetra = tetrahedralizar_y_extraer_superficie(triang_1)
visualizar_transparente_con_bordes(malla_tetra, "Tetraedros 1")

# Exportar cada triángulo como pieza
exportar_triangulos(triang_1, carpeta="triang_1_export")


In [3]:
print("Triangulación 1 :", len(triang_1.faces), "triángulos")

NameError: name 'triang_1' is not defined

<center>